In [ ]:
from __future__ import division
%pylab inline
from scipy import stats
from angles import r2d, r2arcs, d2arcs, arcs2r
def arcm2r(theta):
    return arcs2r(theta*60)
import seaborn as sns;sns.set_style('darkgrid')
import lsst.sims.maf.stackers as stackers
import treecorr as tr
%config InlineBackend.figure_format = 'retina'
matplotlib.rcParams['figure.figsize'] = 19,14
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.metricBundles as metricBundles
import lsst.sims.maf.db as db
from collections import defaultdict
import treecorr
from itertools import chain
sns.set_context('poster',font_scale=1.4)

In [ ]:
outDir='temp'
conn = db.OpsimDatabaseV4('simruns/feature_baseline_update_10yrsv2')
bundleList = []
sql = 'filter = "r" and proposalId = 0 and airmass>0'
metric = metrics.CountMetric('airmass')
slicer = slicers.HealpixSlicer()
#dith_stack = stackers.NormAirmassStacker()
bundleList.append(metricBundles.MetricBundle(metric, slicer, sql))#, stackerList=[dith_stack]))
bd = metricBundles.makeBundlesDictFromList(bundleList)
mbg = metricBundles.MetricBundleGroup(bd, conn, outDir=outDir, saveEarly= False)
mbg.runAll()
conn.close()



In [ ]:
num, xedges, yedges = np.histogram2d(mbg.simData['fieldRA'], mbg.simData['fieldDec'], bins=15)

In [ ]:
H, xedges, yedges = np.histogram2d(mbg.simData['fieldRA'], mbg.simData['fieldDec'], weights=mbg.simData['airmass'], bins=15)
H = H.T
plt.imshow(H/num.T, interpolation='cubic', origin='low', extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
plt.colorbar()

- query the database for filter, dec, airmass
- for each filter: plot distribution of airmasses histogram overall
- for each filter: for each declination band: plot airmass histogram


In [ ]:
outDir='temp'
conn = db.OpsimDatabaseV4('simruns/feature_baseline_update_10yrsv2')
bundleList = []
sql = 'proposalId = 0 and airmass>0'
metric = metrics.CountMetric('filter')
slicer = slicers.HealpixSlicer()
dith_stack = stackers.NormAirmassStacker()
bundleList.append(metricBundles.MetricBundle(metric, slicer, sql, stackerList=[dith_stack]))
bd = metricBundles.makeBundlesDictFromList(bundleList)
mbg = metricBundles.MetricBundleGroup(bd, conn, outDir=outDir, saveEarly= False)
mbg.runAll()
conn.close()

In [ ]:
filter_data = mbg.simData['filter']
dec_data = mbg.simData['fieldDec']
airmass_data = mbg.simData['airmass']

In [ ]:
from collections import namedtuple
ks_tuple = namedtuple('ks_tuple', ['Dstatistic', 'pvalue'])

In [ ]:
airmass_dict

In [ ]:
ks_stats_dict

In [ ]:
norm_airmass = airmass_data/min(airmass_data)
filters = ['u','g','r','i','z','y']
ks_stats_dict = {k:[] for k in filters}
airmass_dict = {k:[] for k in filters}
for dec_band, dec_band_next in zip([0,-20,-40,-60,-80],[20,0,-20,-40,-60]):
    airmass_per_dec_band = airmass_data[np.logical_and(dec_data>dec_band, dec_data<dec_band_next)]
    plt.hist(airmass_per_dec_band, label='average', alpha=0.2, bins=100)
    for filt in filters:
        
        plt.axvline(2)
        plt.axvline(1)
        filtered_airmass = airmass_data[np.logical_and(np.logical_and(filter_data==filt,dec_data>dec_band),dec_data<dec_band_next)]
        #filtered_airmass /= min(filtered_airmass)
        plt.hist(filtered_airmass, label=filt+str(dec_band), histtype='step', linewidth=2.5, bins=100)
        airmass_dict[filt].append(filtered_airmass)
        ks_test = stats.ks_2samp(filtered_airmass, airmass_per_dec_band)
        ks_stats_dict[filt].append(ks_tuple(Dstatistic=ks_test.statistic, pvalue=ks_test.pvalue))
    plt.legend()
    plt.xscale('log')
    plt.xlim(0,2)
    plt.yscale('log')
    plt.figure()

In [ ]:
from functools import reduce
ra_data = mbg.simData['fieldRA']
filters = ['u','g','r','i','z','y']
dec_bands = [0,-10,-20,-30,-40,-50,-60,-70]

In [ ]:
ks_stats_dict = {k:{dec:[] for dec in dec_bands} for k in filters}
for dec_band, dec_band_next in zip([0,-10,-20,-30,-40,-50,-60,-70],
                                   [10,0,-10,-20,-30,-40,-50,-60]):
    for filt in filters:
        airmass_perfilter_perdec = airmass_data[reduce(np.logical_and,
                                                       (filter_data==filt,
                                                        dec_data>dec_band,
                                                        dec_data<dec_band_next))]
        for ra_band, ra_band_next in zip(np.linspace(0,350,36),np.linspace(10,360,36)):
            filtered_airmass = airmass_data[reduce(np.logical_and,
                                                   (filter_data==filt,
                                                    dec_data>dec_band,
                                                    dec_data<dec_band_next, 
                                                    ra_data>ra_band, 
                                                    ra_data<ra_band_next))]
            ks_test = stats.ks_2samp(filtered_airmass, airmass_perfilter_perdec)
            if ks_test.pvalue<10**-10:
                pval = 10**-10
            else:
                pval = ks_test.pvalue
            ks_stats_dict[filt][dec_band].append(ks_tuple(Dstatistic=ks_test.statistic, 
                                                          pvalue=pval))

In [ ]:
ks_stats_dict

In [ ]:
for dec in dec_bands:
    plt.figure()
    plt.title(str(dec))
    for filt in filters:
        pvals = [i.pvalue for i in ks_stats_dict[filt][dec]]
        plt.hist(pvals, alpha=0.2, linewidth=2.5, label=filt, bins=np.logspace(-10,0,20))
        plt.xscale('log')
        plt.legend()
        plt.gca().set_xlim(right=2)
        plt.yscale('log')